# Qiita ~ 傾向スコアでセレクションバイアスを補正する~

In [1]:
import numpy as np
import pandas as pd

from pandas import DataFrame

from sklearn.metrics import roc_auc_score
import statsmodels.api as sm

### RHC Data

In [2]:
# データの読み込み
data_df = pd.read_csv("http://biostat.mc.vanderbilt.edu/wiki/pub/Main/DataSets/rhc.csv")
#data_df.head()

In [3]:
data_df.head()

,Unnamed: 0,cat1,cat2,ca,sadmdte,dschdte,dthdte,lstctdte,death,cardiohx,...,meta,hema,seps,trauma,ortho,adld3p,urin1,race,income,ptid
0,1,COPD,NaN,Yes,11142,11151.0,NaN,11382,No,0,...,No,No,No,No,No,0.0,NaN,white,Under $11k,5
1,2,MOSF w/Sepsis,NaN,No,11799,11844.0,11844.0,11844,Yes,1,...,No,No,Yes,No,No,NaN,1437.0,white,Under $11k,7
2,3,MOSF w/Malignancy,MOSF w/Sepsis,Yes,12083,12143.0,NaN,12400,No,0,...,No,No,No,No,No,NaN,599.0,white,$25-$50k,9
3,4,ARF,NaN,No,11146,11183.0,11183.0,11182,Yes,0,...,No,No,No,No,No,NaN,NaN,white,$11-$25k,10
4,5,MOSF w/Sepsis,NaN,No,12035,12037.0,12037.0,12036,Yes,0,...,No,No,No,No,No,NaN,64.0,white,Under $11k,11


In [4]:
# 死亡率とRHCの有無のクロス集計
pd.crosstab(data_df.death, data_df.swang1)

swang1,No RHC,RHC
death,,
No,1315,698
Yes,2236,1486


In [5]:
# RHCの有無での死亡率の差を計算
(1486 / (698 + 1486)) - (2236 / (1315 + 2236))

0.050721150622586864

In [6]:
# 用いる説明変数群、詳細はデータセットの記述を参照
cols = ["cat1", "sex", "race", "edu", "income",
        "resp", "card", "neuro", "gastr", "renal", "meta", "hema", "seps", "trauma", "ortho",
        "das2d3pc", "dnr1", "ca", "surv2md1", "aps1", "scoma1", "wtkilo1", "temp1",
        "resp1", "hrt1", "pafi1", "paco21", "ph1", "wblc1", "hema1", "sod1", "pot1", "crea1",
        "bili1", "alb1", "cardiohx", "chfhx", "dementhx", "psychhx", "chrpulhx", "renalhx",
        "liverhx", "gibledhx", "immunhx", "transhx", "amihx",
        "age", "meanbp1"]

# 説明変数中のカテゴリカル変数
categorical_columns = ["cat1", "sex", "race", "edu", "income", "ca", "dnr1",
                       "resp", "card", "neuro", "gastr", "renal", "meta", "hema", "seps", "trauma", "ortho"]

# カテゴリカル変数のダミー化
data_df.loc[:, "Intercept"] = 1
X = data_df[cols + ["Intercept"]]
dummy = pd.get_dummies(X[categorical_columns])
X = pd.concat([X, dummy], axis=1).drop(categorical_columns, axis=1).values


# RHC有無のダミー変数
w1 = pd.get_dummies(data_df["swang1"])["RHC"].values

# 目的変数
y = pd.get_dummies(data_df["death"])["Yes"].values

In [7]:
# StatsModelsのLogitにより傾向スコアを推定
glm = sm.Logit(w1, X)
result = glm.fit()
ps1 = result.predict(X)

         Current function value: inf
         Iterations: 35


/opt/anaconda3/lib/python3.6/site-packages/statsmodels/discrete/discrete_model.py:1724: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))
/opt/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


In [8]:
# c統計量としてAUCをを計算
roc_auc_score(w1, ps1)

0.7963133740379587

In [9]:
# IPWによりATEを推定
ipwe1 = np.sum((w1 * y) / ps1) / np.sum(w1 / ps1)
ipwe0 = np.sum(((1 - w1) * y) / (1.0 - ps1)) / np.sum((1 - w1) / (1.0 - ps1))
ATE = ipwe1 - ipwe0
ATE

0.059075127898833735

### CM Data

In [10]:
# データの読み込み
data_df = pd.read_csv('https://github.com/iwanami-datascience/vol3/raw/master/kato%26hoshino/q_data_x.csv')
data_df.head()

,cm_dummy,gamedummy,area_kanto,area_keihan,area_tokai,area_keihanshin,age,sex,marry_dummy,job_dummy1,...,T,F1,F2,F3,M1,M2,M3,TVwatch_day,gamesecond,gamecount
0,0,0,0,0,0,1,44.5,1,1,1,...,0,0,0,0,0,1,0,33.427600,0,0
1,0,0,0,1,0,0,34.5,1,1,1,...,0,0,0,0,0,1,0,31.542862,0,0
2,0,0,0,1,0,0,24.5,1,0,0,...,0,0,0,0,1,0,0,37.825805,0,0
3,0,0,0,1,0,0,44.5,1,1,1,...,0,0,0,0,0,1,0,36.345911,0,0
4,0,0,0,1,0,0,34.5,1,1,1,...,0,0,0,0,1,0,0,49.344942,0,0


In [11]:
# CM視聴有無でのアプリ利用率の差を計算
data_df[data_df.cm_dummy == 1].gamedummy.mean() - data_df[data_df.cm_dummy == 0].gamedummy.mean()

0.002202143595586223

In [12]:
# CM視聴有無でのアプリ利用回数の差を計算
data_df[data_df.cm_dummy == 1].gamecount.mean() - data_df[data_df.cm_dummy == 0].gamecount.mean()

-1.4845493913116865

In [13]:
# CM視聴有無でのアプリ利用時間の差を計算
data_df[data_df.cm_dummy == 1].gamesecond.mean() - data_df[data_df.cm_dummy == 0].gamesecond.mean()

-629.6405765396544

In [14]:
# 説明変数
cols = ["age", "sex", "TVwatch_day", "marry_dummy", "child_dummy", "inc", "pmoney",
        "area_kanto", "area_tokai", "area_keihanshin",
        "job_dummy1", "job_dummy2", "job_dummy3", "job_dummy4", "job_dummy5", "job_dummy6",
        "fam_str_dummy1", "fam_str_dummy2", "fam_str_dummy3", "fam_str_dummy4"]
data_df["intercept"] = 1
X = data_df[cols + ["intercept"]].values

# CM視聴有無ダミー
w2 = data_df.cm_dummy.values

# 目的変数群（1:アプリ利用ダミー, 2:アプリ利用回数、3:アプリ利用時間）
y1 = data_df.gamedummy.values
y2 = data_df.gamecount.values
y3 = data_df.gamesecond.values

In [15]:
# StatsModelsのLogitにより傾向スコアを推定
glm = sm.Logit(w2, X)
result = glm.fit()
ps2 = result.predict(X)

Optimization terminated successfully.
         Current function value: 0.542152
         Iterations 6


In [16]:
# c統計量としてAUCをを計算
roc_auc_score(w2, ps2)

0.7917012811992321

In [17]:
# IPWによりアプリ利用ダミーへのATEを推定
ipwe11 = np.sum((w2 * y1) / ps2) / np.sum(w2 / ps2)  # Treated
ipwe10 = np.sum(((1 - w2) * y1) / (1.0 - ps2)) / np.sum((1 - w2) / (1.0 - ps2))  # Control
ATE1 = ipwe11 - ipwe10
ATE1

0.03231177330512102

In [18]:
# IPWによりアプリ利用回数へのATEを推定
ipwe21 = np.sum((w2 * y2) / ps2) / np.sum(w2 / ps2)  # Treated
ipwe20 = np.sum(((1 - w2) * y2) / (1.0 - ps2)) / np.sum((1 - w2) / (1.0 - ps2))  # Control
ATE2 = ipwe21 - ipwe20
ATE2

5.3490295664746235

In [19]:
# IPWによりアプリ利用時間へのATEを推定
ipwe31 = np.sum((w2 * y3) / ps2) / np.sum(w2 / ps2)
ipwe30 = np.sum(((1 - w2) * y3) / (1 - ps2)) / np.sum((1 - w2) / (1 - ps2))
ATE3 = ipwe31 - ipwe30
ATE3

1513.69969078252